# Scaling tendon slack length and optimal fibre length

In [ ]:
import numpy as np
import os
import sys
import opensim as osim
import pickle
import tkinter as tk
from tkinter import filedialog

sys.path.insert(1, r'Functions')  # add to path

from optimal_muscle_parameters import optimal_muscle_parameters
from assess_muscle_param_var import assess_muscle_param_var
from assess_muscle_mapping import assess_muscle_mapping

This script calculates the tendon slack length and optimal fibre length for a kinematic model.

Requirements: a subject specific scaled model (e.g., __AB08.osim__) and the generic base model (e.g., __gait2392_simbody.osim__)

In [ ]:
root = tk.Tk()
root.withdraw()

osim_model_ref_filepath = filedialog.askopenfilename(initialdir = os.getcwd(),title = "Select reference model",filetypes = (("osim files","*.osim"),("all files","*.*")))
osim_model_targ_filepath = filedialog.askopenfilename(initialdir = os.getcwd(),title = "Select scaled model",filetypes = (("osim files","*.osim"),("all files","*.*")))

# Folders used
optimsed_model_folder = filedialog.askdirectory(initialdir = os.getcwd(),title = "Select optimised model folder (OUTPUT)")
results_folder = filedialog.askdirectory(initialdir = os.getcwd(),title = "Select results folder (OUTPUT)")
log_folder = optimsed_model_folder

# Check if results and optimised model folder exists
if not os.path.exists(optimsed_model_folder):
    os.makedirs(optimsed_model_folder)

if not os.path.exists(results_folder):
    os.makedirs(results_folder)

# Reference model for calculating results metrics
osim_model_ref = osim.Model(osim_model_ref_filepath)

In [ ]:
N_eval = int(input("Enter number of evaluations to perform: ")) 

''' Muscle optimiser '''
# Optimising target model based on reference model for N_eval points per degree of freedom
osim_model_opt, sim_info = optimal_muscle_parameters(osim_model_ref_filepath, osim_model_targ_filepath, N_eval, log_folder)

# Setting the output folder
if optimsed_model_folder == '' or len(optimsed_model_folder) == 0:
    optimsed_model_folder = targ_model_folder

# Printing the optimised model
osim_model_opt.printToXML(os.path.join(optimsed_model_folder, osim_model_opt.getName()));

### Saving Results

At this point, the newly optimised model has already been saved. This section is for analysis to find the optimal number of evaluations to perform.

If you only need the optimised model (for tendon slack length and optimal fibre length) and are happy with the number of evaluations, feel free to skip this last step. In the chosen 'optimised model folder', there is a log folder which contains the optimised parameters if required to know.

In [ ]:
''' Saving Results '''
# Variation in muscle parameters
results_musvarmetrics = assess_muscle_param_var(osim_model_ref, osim_model_opt, N_eval, results_folder)
# Assess muscle mapping (RMSE, max error, etc) at n_Metrics points between reference and optimised model
n_Metrics = 10
results_musmapmetrics = assess_muscle_mapping(osim_model_ref, osim_model_opt, N_eval, n_Metrics, results_folder)

# Save simulation info
with open(results_folder + "\\SimsInfo" + '.pckl', 'wb') as f:
    pickle.dump([sim_info], f)